# LightGBM

# Feature Engineering

In [69]:
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error,r2_score
import time

import lightgbm as lgb
from lightgbm import LGBMRegressor
from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings("ignore")

In [70]:
"""
Contributions from:
DSEverything - Mean Mix - Math, Geo, Harmonic (LB 0.493) 
https://www.kaggle.com/dongxu027/mean-mix-math-geo-harmonic-lb-0-493
JdPaletto - Surprised Yet? - Part2 - (LB: 0.503)
https://www.kaggle.com/jdpaletto/surprised-yet-part2-lb-0-503
hklee - weighted mean comparisons, LB 0.497, 1ST
https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st

Also all comments for changes, encouragement, and forked scripts rock

Keep the Surprise Going
"""

import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime

data = {
    'tra': pd.read_csv('../../../mltestdata/05_recruit/air_visit_data.csv'),
    'as': pd.read_csv('../../../mltestdata/05_recruit/air_store_info.csv'),
    'hs': pd.read_csv('../../../mltestdata/05_recruit/hpg_store_info.csv'),
    'ar': pd.read_csv('../../../mltestdata/05_recruit/air_reserve.csv'),
    'hr': pd.read_csv('../../../mltestdata/05_recruit/hpg_reserve.csv'),
    'id': pd.read_csv('../../../mltestdata/05_recruit/store_id_relation.csv'),
    'tes': pd.read_csv('../../../mltestdata/05_recruit/sample_submission.csv'),
    'hol': pd.read_csv('../../../mltestdata/05_recruit/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }

data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

for df in ['ar','hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    tmp1 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

#sure it can be compressed...
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = data['tra'].groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, data['as'], how='left', on=['air_store_id']) 
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
lbl = preprocessing.LabelEncoder()
for i in range(10):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date']) 
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date']) 

train = pd.merge(train, stores, how='left', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

# NEW FEATURES FROM JMBULL
train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

# NEW FEATURES FROM Georgii Vyshnia
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2'] = lbl.transform(test['air_store_id'])

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

In [71]:
print( "\nProcessing data for LightGBM ..." )
for c, dtype in zip(train.columns, train.dtypes):	
    if dtype == np.float64:		
        train[c] = train[c].astype(np.float32)

for c, dtype in zip(test.columns, test.dtypes):	
    if dtype == np.float64:		
        test[c] = test[c].astype(np.float32)


Processing data for LightGBM ...


In [72]:
drop_cols=['visitors','air_store_id','visit_date','id']
y_train=train['visitors']
x_train=train.drop(drop_cols, axis=1)

x_test=test.copy()
x_test=x_test.drop(drop_cols, axis=1)

# Modeling

In [73]:
y = train.visitors
train_input = train.copy()
test_input = test.copy()

drop_cols=['visitors','air_store_id','visit_date','id']
train_input=train_input.drop(drop_cols, axis=1)
test_input=test_input.drop(drop_cols, axis=1)

In [74]:
localtrain, localval = train_test_split(train,test_size=0.3,random_state=2018)

y_localtrain=localtrain['visitors']
x_localtrain=localtrain.drop(drop_cols, axis=1)

y_localval=localval['visitors']
x_localval=localval.drop(drop_cols, axis=1)

## LightGBM

In [75]:
#Define a evaluation function

def eval_rmsle(preds, true):
    eval_rmsle = (np.sum((np.log1p(preds)-np.log1p(true))**2)/len(true))**0.5
    return eval_rmsle

#def eval_rmsle(y, pred):
#    return mean_squared_error(y, pred)**0.5

def auc_to_gini_norm(auc_score):
    return 2*auc_score-1

def eval_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [76]:
#Define a evaluation matrix 
from sklearn.metrics.scorer import make_scorer
RMSLE = make_scorer(eval_rmsle)

In [77]:
# Define a function for comparing predictions and true data.
def compare_result(preds, true):
    compare = pd.DataFrame({"test_id": true.index,
                           "real_cost": true,
                           "pred_cost": preds})
    compare = compare[["test_id", "real_cost", "pred_cost"]].reset_index(drop=True)
    
    compare["error_percent_(%)"] = np.abs(compare.real_cost - compare.pred_cost) / compare.real_cost * 100
    
    return compare

### LGBM Regression and GridSearch

In [78]:
# =================================================
# model_selection.GridSearchCV
# =================================================

# 1) Create model
#set static parameters
params = {'boosting_type': 'dart',
          'max_depth' : 5,
          'max_bin' : 500,
          'learning_rate': 0.1,  # 0.618580
          'num_leaves': 22,
         }

#define XGB Grid Search model
model = LGBMRegressor(boosting_type=params['boosting_type']
                       )
#set parameters
gridParams = {
    'colsample_bytree' :[0.9, 1.0], #0.6, 0.64, 0.85 /0.8, 
    'learning_rate': [0.075,0.1], # ,0.005, 0.06/0.05,
    'max_depth' :[2,5,10], #2,3,4
    'n_estimators': [8,24], #,100
    #'num_leaves': [10,20], #2,4,5,6, 100
    'objective': ['binary','regression'],
    'reg_alpha' : [0.1,0.5], #,1.2
    'reg_lambda' : [1.3,1.5], #1, 1.2,1.4
    'subsample' :[0.7,0.9], #0.75 
}

# 3) Run GridSearch
grid = GridSearchCV(model, gridParams, verbose=1, cv=5, n_jobs=-1, scoring=RMSLE)

In [ ]:
#grid search experiment
start = time.time()

grid.fit(x_localtrain, y_localtrain)

end = time.time()
duration = end-start
print ("It takes {} seconds".format(duration))

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


In [12]:
#get/show the best parameters
best_parameters, score, _ = min(grid.grid_scores_, key=lambda x: x[1])
print('score:', score)

for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

score: 0.561887739802
colsample_bytree: 1.0
learning_rate: 0.1
max_depth: 10
n_estimators: 24
objective: 'regression'
reg_alpha: 0.5
reg_lambda: 1.3
subsample: 0.9


In [38]:
#use best model to predict
start = time.time()
lgb_regressor = LGBMRegressor(
    colsample_bytree=best_parameters["colsample_bytree"],
    learning_rate=best_parameters["learning_rate"],
    max_depth=best_parameters["max_depth"],
    n_estimators=best_parameters["n_estimators"],
    objective=best_parameters["objective"],
    reg_alpha=best_parameters["reg_alpha"],
    reg_lambda=best_parameters["reg_lambda"],
    subsample=best_parameters["subsample"]
)

y_localtrain_log = np.log1p(y_localtrain)

model = lgb_regressor.fit(x_localtrain, y_localtrain_log)
y_localvalprediction1 = model.predict(x_localval)

y_localvalprediction = np.expm1(y_localvalprediction1)

rmsle_lgb = eval_rmsle(y_localvalprediction, y_localval)
print ("LGBM RMSLE is : {}".format(rmsle_lgb))

end = time.time()
duration = end - start
print ("It takes {} seconds".format(duration))

compare_lgb = compare_result(preds=y_localvalprediction, true=y_localval)

LGBM RMSLE is : 0.5120897445327701
It takes 1.2458980083465576 seconds


In [41]:
compare_lgb.head(10)

,test_id,real_cost,pred_cost,error_percent_(%)
0,236487,26,18.749872,27.885106
1,181786,29,25.677078,11.458353
2,163879,58,39.483905,31.924302
3,22266,4,10.235186,155.879645
4,229422,53,17.266205,67.422255
5,188981,22,21.267665,3.328795
6,239241,35,41.632357,18.949592
7,158829,22,21.575819,1.928094
8,38316,1,18.007237,1700.723660
9,97073,1,4.998981,399.898067


# Cross validation with LightGBM

In [26]:
def cross_validate_lgb(params, x_train, y_train, kf, verbose=True, verbose_eval=50):

    start_time = time.time()
    train_pred = np.zeros((x_train.shape[0]))
    #test_pred = np.zeros((x_test.shape[0]))

    #if len(cat_cols)==0: use_cat=False

    # use the k-fold object to enumerate indexes for each training and validation fold
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)): # folds 1, 2 ,3 ,4, 5
        # example: training from 1,2,3,4; validation from 5
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]
        y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]


        # Might not be required in this competition
        #if use_cat:
        #    lgb_train = lgb.Dataset(x_train_kf, y_train_kf, categorical_feature=cat_cols)
        #    lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train, categorical_feature=cat_cols)
        #else:
        lgb_train = lgb.Dataset(x_train_kf, y_train_kf)
        lgb_val = lgb.Dataset(x_val_kf, y_val_kf, reference=lgb_train)
        
        #watchlist= [(lgb_train, "train"), (lgb_val, 'val')]
        watchlist= [lgb_train, lgb_val]

        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=4000,
                        #valid_sets=lgb_val,
                        valid_sets=watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=verbose_eval,
                        #feval=eval_rmsle
                       )

        val_pred = gbm.predict(x_val_kf)

        # Might not be required in this competition
        #if use_rank:
        #    train_pred[val_index] += probability_to_rank(val_pred)
        #    test_pred += probability_to_rank(gbm.predict(x_test))
        #    # test_pred += gbm.predict(x_test)
        #else:
        train_pred[val_index] += val_pred
        #test_pred += gbm.predict(x_test)
            
        #fold_auc = roc_auc_score(y_val_kf.values, val_pred)
        #fold_gini_norm = auc_to_gini_norm(fold_auc)
        fold_rmsle = eval_rmsle(val_pred, y_val_kf.values)

        if verbose:
            #print('fold cv {} AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(i, fold_auc, fold_gini_norm))
            print('fold cv {} RMSLE score is {:.6f}'.format(i, fold_rmsle))

    #test_pred /= kf.n_splits

    #cv_auc = roc_auc_score(y_train, train_pred)
    #cv_gini_norm = auc_to_gini_norm(cv_auc)
    cv_rmsle = eval_rmsle(train_pred, y_train)
    #cv_score = [cv_auc, cv_gini_norm]
    #cv_score = [cv_rmsle]
    if verbose:
        #print('cv AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(cv_auc, cv_gini_norm))
        print('cv RMSLE score is {:.6f}'.format(cv_rmsle))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))

    #return cv_score, train_pred,test_pred
    return cv_rmsle

let's test the above cross validation function with some simple lgbm parameters

In [27]:
lgb_params = {
    'boosting_type': 'dart',
    'max_depth' : 5,
    'max_bin' : 500,
    'learning_rate': 0.1,  # 0.618580
    'num_leaves': 22,
    'metric': 'RMSE'
}

# only do 3 fold CV here so that we save some running time on Kaggle Kernel
kf=StratifiedKFold(n_splits=3, shuffle=True, random_state=2018)

print('Start training...')

#cv_score =cross_validate_xgb(xgb_params, train_input, y, kf, verbose=False, verbose_eval=50)
cv_score =cross_validate_lgb(lgb_params, train_input, y, kf, verbose=False, verbose_eval=50)

print('cv score is {:.6f}'.format(cv_score))

Start training...
Training until validation scores don't improve for 30 rounds.
[50]	training's rmse: 11.3071	valid_1's rmse: 10.5027
Early stopping, best iteration is:
[45]	training's rmse: 11.0605	valid_1's rmse: 10.2589
Training until validation scores don't improve for 30 rounds.
[50]	training's rmse: 10.2892	valid_1's rmse: 12.5188
Early stopping, best iteration is:
[45]	training's rmse: 10.0179	valid_1's rmse: 12.2622
Training until validation scores don't improve for 30 rounds.
[50]	training's rmse: 11.4438	valid_1's rmse: 10.1763
Early stopping, best iteration is:
[45]	training's rmse: 11.1957	valid_1's rmse: 9.92261
cv score is 0.605949


# Bayesian Optimsation - Setup

In [61]:
params={
    'num_leaves':(2,30),
    'min_sum_hessian_in_leaf':(2,30),
    'max_depth':(2,30),
    'feature_fraction':(0.0,1.0),
    'bagging_fraction':(0.0,1.0),
#    'max_drop':(1,20),
#    'drop_rate':(0.0,1.0),
#    'min_data_in_leaf':(2,30),
    'bagging_freq':(0,2),
    'lambda_l1':(0.0,1.0)
#    'lambda_l2':(0.0,1.0),
#    'n_estimators':(2,30), 
#    'colsample_bytree':(0.4, 1),
#    'reg_lambda':(0.0,2.0)
}

In [66]:
# reload(lgb_wrapper)
#def lgbcv_func(max_depth, learning_rate, subsample, colsample_bytree, nthread=4, seed=0):
def lgbcv_func(num_leaves, min_sum_hessian_in_leaf, max_depth, 
               feature_fraction, bagging_fraction, bagging_freq, lambda_l1, nthread=4, seed=0):

    params = {
        'objective' : "regression",
        'learning_rate': 0.1,       
        'task': 'train',
        'boosting_type': 'dart',
        'metric': 'RMSE'
    }
    
    # for a more ideal out-of-fold model prediction for this dataset, we use 10-fold CV
    kf=StratifiedKFold(n_splits=3, shuffle=True, random_state=2018)
    
    # we will disable all the verbose setting in this functional call, so that we don't have too much information 
    # to read during the bayesian optimisation process.
    return 1-cross_validate_lgb(params, train_input, y, kf, verbose=False, verbose_eval=False)

In [67]:
lgb_bo=BayesianOptimization(lgbcv_func, params)

In [68]:
lgb_bo.maximize(init_points=5, n_iter=20)

Initialization
---------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   bagging_fraction |   bagging_freq |   feature_fraction |   lambda_l1 |   max_depth |   min_sum_hessian_in_leaf |   num_leaves | 
    1 | 00m09s |    0.39466 |             0.3508 |         1.3121 |             0.1093 |      0.2764 |     11.6430 |                   24.7519 |      25.8416 | 
    2 | 00m09s |    0.39466 |             0.8101 |         0.5614 |             0.9704 |      0.6502 |      5.0086 |                    7.4378 |      19.8911 | 
    3 | 00m09s |    0.39466 |             0.4453 |         0.9001 |             0.3496 |      0.2525 |      9.2718 |                    9.0385 |      17.8205 | 
    4 | 00m09s |    0.39466 |             0.3424 |         1.4994 |             0.9922 |      0.8231 |      8.2559 |                   12.7862 |       9.0365 | 
    5 | 00m10s |    

In [ ]:

    'subsample' : 
    'reg_alpha' : 
    'min_split_gain' :[],
    'subsample_for_bin' :[],
    'max_drop' :[], 
    'gaussian_eta' :[], 
    'drop_rate' :[],
    'silent' :[], 
    'boosting_type' :['gbdt'], 
    'min_child_weight' :[], 
    'skip_drop' :[], 
    'fair_c' :[], 
    'seed' :[], 
    'poisson_max_delta_step' :[], 
    'subsample_freq' :[], 
    'max_bin' :[],  #55
    'nthread' :[], 
    'min_child_samples' :[], 
    'huber_delta' :[], 
    'use_missing' :[], 
    'uniform_drop' :[], 
    'bagging_fraction': [] #0.8,
    'bagging_freq': [] # 5
    'feature_fraction': [] # 0.2319,
    'feature_fraction_seed': [] #9
    'bagging_seed': [] #9,
    'min_data_in_leaf': [] #6
    'min_sum_hessian_in_leaf': [] # 11                              
    'xgboost_dart_mode' :[]

    'max_depth':(4,10),
    'learning_rate':(0.05,0.3),
    'subsample': (0.4, 1),
    'colsample_bytree': (0.4, 1)

In [ ]:
score: 0.561887739802
colsample_bytree: 1.0
learning_rate: 0.1
max_depth: 10
n_estimators: 24
objective: 'regression'
reg_alpha: 0.5
reg_lambda: 1.3
subsample: 0.9

** Cross validation **

In [54]:
from sklearn.model_selection import StratifiedKFold,KFold

### Kfold or Stratified Kfold
kf=KFold(n_splits=5, shuffle=True, random_state=2017)
#kf=StratifiedKFold(n_splits=5, shuffle=True, random_state=2017)

### Criterion function
def auc_to_gini_norm(auc_score):
    return 2*auc_score-1

def eval_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def eval_rmsle(y, pred):
    return mean_squared_error(y, pred)**0.5

In [142]:
### Modified cross-validation function copied from Yifan's cool kernel. 
### Source: https://www.kaggle.com/yifanxie/porto-seguro-tutorial-end-to-end-ensemble

def cross_validate_sklearn(clf, x_train, y_train, x_test, kf, scale=False, verbose=True):
    start_time=time.time()
    
    # initialise the size of out-of-fold train an test prediction
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))

    # use the kfold object to generate the required folds
    for i, (train_index, test_index) in enumerate(kf.split(x_train, y_train)):
        # generate training folds and validation fold
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[test_index, :]
        y_train_kf, y_val_kf = y_train[train_index], y_train[test_index]

        # perform scaling if required i.e. for linear algorithms
        if scale:
            scaler = StandardScaler().fit(x_train_kf.values)
            x_train_kf_values = scaler.transform(x_train_kf.values)
            x_val_kf_values = scaler.transform(x_val_kf.values)
            x_test_values = scaler.transform(x_test.values)
        else:
            x_train_kf_values = x_train_kf.values
            x_val_kf_values = x_val_kf.values
            x_test_values = x_test.values
        
        ###################################################
        # Please update for your own criterion 
        ###################################################
        # fit the input classifier and perform prediction.
        #clf.fit(x_train_kf_values, y_train_kf.values)
        clf.fit(x_train_kf_values, np.log1p(y_train_kf.values))

        ###################################################
        # Please update for your own criterion 
        ###################################################
        #val_pred=clf.predict_proba(x_val_kf_values)[:,1]
        val_pred=clf.predict(x_val_kf_values)

        train_pred[test_index] += val_pred

        ###################################################
        # Please update for your own criterion 
        ###################################################
        #y_test_preds = clf.predict_proba(x_test_values)[:,1]
        y_test_preds = clf.predict(x_test_values)
        test_pred += y_test_preds

        ###################################################
        # Please update for your own criterion 
        ###################################################
        #fold_auc = roc_auc_score(y_val_kf.values, val_pred)
        #fold_gini_norm = auc_to_gini_norm(fold_auc)
        fold_rmsle = eval_rmsle(np.log1p(y_val_kf.values), val_pred)

        if verbose:
            ###################################################
            # Please update for your own criterion
            ###################################################

            #print('fold cv {} AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(i, fold_auc, fold_gini_norm))
            print('fold cv {} RMSLE score is {:.6f}'.format(i, fold_rmsle))

    test_pred /= kf.n_splits

    ###################################################
    # Please update for your own criterion
    ###################################################
    #cv_auc = roc_auc_score(y_train, train_pred)
    #cv_gini_norm = auc_to_gini_norm(cv_auc)
    cv_rmsle = eval_rmsle(np.log1p(y_train), train_pred)
        
    #cv_score = [cv_auc, cv_gini_norm]
    cv_score = [cv_rmsle]
    if verbose:

        ###################################################
        # Please update for your own criterion
        ###################################################
        #print('cv AUC score is {:.6f}, Gini_Norm score is {:.6f}'.format(cv_auc, cv_gini_norm))
        print('cv RMSLE score is {:.6f}'.format(cv_rmsle))
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
    return cv_score, train_pred, test_pred


In [143]:
rfr = RandomForestRegressor(criterion='mse',
                            max_depth=5,
                            n_estimators=500,
                            min_samples_leaf=5,
                            min_samples_split=2
                           )

outcomes =cross_validate_sklearn(rfr, x_train, y_train ,x_test, kf, scale=False, verbose=True)

fold cv 0 RMSLE score is 0.519012
fold cv 1 RMSLE score is 0.519113
fold cv 2 RMSLE score is 0.516890
fold cv 3 RMSLE score is 0.515128
fold cv 4 RMSLE score is 0.514956
cv RMSLE score is 0.517023
it takes 1131.644 seconds to perform cross validation
